In [82]:
# selenium 불러오기 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime, timedelta 
from datetime import date as dt
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
#driver = webdriver.Chrome(executable_path=r"C:\webdrivers\chromedriver.exe")

In [83]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)
# TBC 접속하기 
driver.implicitly_wait(3)
driver.get('https://www.tbc.net.tw/Epg/')
wait = WebDriverWait(driver, 50)

<ipython-input-83-adcc350ab05f>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)


In [85]:
def update_date(driver,start_date): #ex)2021/02/28
  
  #change hour to initial 00:00
  hour_pick = Select(driver.find_element_by_id("SearchParameter_Hour"))
  hour_pick.select_by_visible_text("00:00") 
  #change date
  date_pick = driver.find_element_by_id('SearchParameter_StartDate')
  upinpt = driver.execute_script("arguments[0].setAttribute('value','{a_date}')".format(a_date = start_date), date_pick)
  #driver.execute_script("($'#SearchParameter_Hour').val('0');")
  #driver.execute_script("($'#SearchParameter_StartDate').val({a_date});".format(a_date = date))
  update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
  # result = driver.execute_script("arguments[0].click();", update_click)
  print('Date is',start_date)
  print('Hour is 00:00',)
  print("Succesfuly updated")
  upclic = driver.execute_script("arguments[0].click();", update_click)
  return (start_date, upinpt,"00:00",upclic)

In [86]:
def next_tmLine(start_hr, start_date,end_date):
  
  opt_seletd = Select(driver.find_element_by_id("SearchParameter_Hour"))
  seletd_tm = opt_seletd.first_selected_option
  print("current time starting "
        "point :{}".format(seletd_tm.text))
  prsent_tm = seletd_tm.text
  if (prsent_tm != "18:00"):
    print("you are eligible to increment hours")
    #to convert string to time format HH:MM
   # start_tm = #"00:00"
    weighted_tm = 6
    int_tm = int(start_hr.rsplit(':', 1)[0])
    added_tm = int_tm + weighted_tm
    start_hr = "{}:00".format(str(added_tm).zfill(2))
    print("Date is",start_date)
    print("new start hour is :",start_hr)
   
    opt_seletd.select_by_visible_text(start_hr) 
    update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
    uphr = driver.execute_script("arguments[0].click();", update_click)
    print("Succesfuly updated")
      
    return (prsent_tm, start_hr,str(start_date), uphr, uphr)
  else:
    print("pls, use update_date_function instead")
    #start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
    start_date = start_date.strftime("%Y/%m/%d")
    #start_date += datetime.timedelta(days=1)
    start_date = str(start_date)
    print("Date is",start_date)
    out = update_date(driver,start_date)
    print("out[0]:",out[0])
    return (out[0],"00:00", out[0])


In [199]:
hr = '00:00'
init_day = '2021/03/01'
end_day = '2021/03/10'
update_date(driver, init_day)
#next_tmLine(hp,tp,fp)
while init_day < end_day:
 get = next_tmLine(hr,init_day,end_day)
 init_day = get[1]
 end_day = get[2]

Date is 2021/03/21
Hour is 00:00
Succesfuly updated
current time starting point :00:00
you are eligible to increment hours
Date is 2021/03/21
new start hour is : 06:00
Succesfuly updated
current time starting point :06:00
you are eligible to increment hours
Date is 2021/03/21
new start hour is : 12:00
Succesfuly updated
current time starting point :12:00
you are eligible to increment hours
Date is 2021/03/21
new start hour is : 18:00
Succesfuly updated
current time starting point :18:00
pls, use update_date_function instead
Date is 2021/03/22
Date is 2021/03/22
Hour is 00:00
Succesfuly updated
out[0]: 2021/03/22
current time starting point :00:00
you are eligible to increment hours
Date is 2021/03/22
new start hour is : 06:00
Succesfuly updated
current time starting point :06:00
you are eligible to increment hours
Date is 2021/03/22
new start hour is : 12:00
Succesfuly updated
current time starting point :12:00
you are eligible to increment hours
Date is 2021/03/22
new start hour is : 

In [87]:
#@markdown check each day info
def today_info():
  """Wait and Check the len of EPG and the group of corresponding CNT """
  try:  
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    if (len(epg) != len(cnt)):  
  # we can pass the message in the raise statement  
      raise ValueError("the number of channels and contents lists don't match!!") 
    else:
      #print('the num of channels and the num of lists of contents on the table match! Keep crawling!') 
      #print("the each num of channel and lists of contents is {}".format(len(epg)))
      print("channel and contents nm {} matches".format(len(epg)))
  except ValueError as e:  
      print(e) 
  return (None)

In [76]:
a_l = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/ul[3]/li[2]/a')))
a_ctn = webdriver.ActionChains(driver).move_to_element(a_l).click()
a_ctn.perform()

In [75]:
element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
element.click()

In [73]:
#@title Create CLOSE btn!!
close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
close_btn_ele = driver.find_element_by_xpath(close_btn)
close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
close_pop.perform()

In [89]:
def get_Epg_info():

    epg_info = []
    
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    for i in range(50): #range(len(epg))
      
      tv_name = epg[i].text.split('\n') #get the channel name
      a_ctn_list = WebDriverWait(cnt[i], 50).until(
          EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
      # equals --> a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#
      ctn_info = []
      for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
        a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click() --》이것을 아래의 element_to_be_clickable로 바뀌기
        a_ctn.perform()
        try: # if one of info is empty, we just leave it as a blank
          driver.implicitly_wait(3)
          ctn_info.append([wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_name"]'))).text] +
                      wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
                      [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_desc"]'))).text.replace("\n", " ")] +
                      [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_date"]'))).text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
                      wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_time"]'))).text.split()
                      )
        
        except (NoSuchElementException, TimeoutException) as e :
          # Just append a None or ""
          ctn_info.append(['']+['']+['']+['']+[''])
        
        epg_info.append({'channel': tv_name,
                          'prog': list(zip(*ctn_info))[0],
                          'genre': list(zip(*ctn_info))[1],
                          'desc': list(zip(*ctn_info))[2],
                          'time': list(zip(*ctn_info))[4]
                        })                                         #//*[@id="epg_popup"]/div/div[1]/div[3]/a
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
        element.click()
        driver.implicitly_wait(3) 
    return (epg_info)

In [61]:
test= get_Epg_info()

In [62]:
test

[{'channel': ['2', '南桃園節目總表'],
  'prog': ('系統資訊',),
  'genre': ('類別：知性（知識）',),
  'desc': ('系統資訊',),
  'time': ('14:00~14:01',)},
 {'channel': ['2', '南桃園節目總表'],
  'prog': ('系統資訊', '數位小百科'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）'),
  'desc': ('系統資訊', '數位小百科'),
  'time': ('14:00~14:01', '14:01~14:48')},
 {'channel': ['2', '南桃園節目總表'],
  'prog': ('系統資訊', '數位小百科', '頻道宣傳'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'desc': ('系統資訊', '數位小百科', '頻道宣傳'),
  'time': ('14:00~14:01', '14:01~14:48', '14:48~15:00')},
 {'channel': ['2', '南桃園節目總表'],
  'prog': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'desc': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊'),
  'time': ('14:00~14:01', '14:01~14:48', '14:48~15:00', '15:00~15:01')},
 {'channel': ['2', '南桃園節目總表'],
  'prog': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊', '數位小百科'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'desc': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊', '數位小百科'),
  'time':

In [90]:
hr = '00:00'
init_day = '2021/03/01'
end_day = '2021/03/10'
update_date(driver, init_day)
#next_tmLine(hp,tp,fp)
EPG_per = []
while init_day < end_day:
 EPG_per.append(get_Epg_info())
 get = next_tmLine(hr,init_day,end_day)
 hr = get[1]
 init_day = get[2]
 

Date is 2021/03/01
Hour is 00:00
Succesfuly updated
current time starting point :00:00
you are eligible to increment hours
Date is 2021/03/01
new start hour is : 06:00
Succesfuly updated
current time starting point :06:00
you are eligible to increment hours
Date is 2021/03/01
new start hour is : 12:00
Succesfuly updated


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=89.0.4389.90)


In [92]:

#@title Save the Genres variables
#save variables
import pickle
with open('EPG_per','wb') as f:
  pickle.dump(EPG_per, f)

In [93]:
EPG_per

[[{'channel': ['2', '南桃園節目總表'],
   'prog': ('系統資訊',),
   'genre': ('類別：知性（知識）',),
   'desc': ('系統資訊',),
   'time': ('00:00~00:01',)},
  {'channel': ['2', '南桃園節目總表'],
   'prog': ('系統資訊', '數位小百科'),
   'genre': ('類別：知性（知識）', '類別：知性（知識）'),
   'desc': ('系統資訊', '數位小百科'),
   'time': ('00:00~00:01', '00:01~00:30')},
  {'channel': ['2', '南桃園節目總表'],
   'prog': ('系統資訊', '數位小百科', '頻道宣傳'),
   'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
   'desc': ('系統資訊', '數位小百科', '頻道宣傳'),
   'time': ('00:00~00:01', '00:01~00:30', '00:30~01:00')},
  {'channel': ['2', '南桃園節目總表'],
   'prog': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊'),
   'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
   'desc': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊'),
   'time': ('00:00~00:01', '00:01~00:30', '00:30~01:00', '01:00~01:01')},
  {'channel': ['2', '南桃園節目總表'],
   'prog': ('系統資訊', '數位小百科', '頻道宣傳', '系統資訊', '數位小百科'),
   'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
   'desc': ('系統資訊', '數位小百科', '頻道宣傳', '系統